In [5]:
from utils.location_functions import predict_future_dist_with_dir_and_speed
import pandas as pd

# Read GNSS data from file
df = pd.read_csv('./data/gnss_dataset.csv')
target_lat = df['latitude'].iloc[-1]
target_long = df['longitude'].iloc[-1]
time_interval = 60  # 60 seconds

direction = 45.15
speed = 12.25 #km/h

# Get the most recent position from the data
current_long = df['longitude'].iloc[0]
current_lat = df['latitude'].iloc[0]


result = predict_future_dist_with_dir_and_speed(
    current_long, current_lat, target_long, target_lat, 
    direction, speed, time_interval
)

future_lon, future_lat, dist_traveled, percent_done = result

print("Prediction Results:")
print(f"Future Position: ({future_lat:.6f}, {future_lon:.6f})")
print(f"Distance Traveled: {dist_traveled:.2f} meters")
print(f"Trip Completion: {percent_done:.2f}%")


Prediction Results:
Future Position: (37.776220, -122.417719)
Distance Traveled: 204.17 meters
Trip Completion: 21.04%


In [6]:
history_20 = df[['timestamp','latitude', 'longitude']].head(20).values.tolist()
history_20

[['2025-05-28 17:29:06.807800', 37.77492445005415, -122.41936732229972],
 ['2025-05-28 17:29:07.807800', 37.77495718265613, -122.41932442741896],
 ['2025-05-28 17:29:08.807800', 37.77498041260144, -122.41930881144076],
 ['2025-05-28 17:29:09.807800', 37.77500200798414, -122.41928100377665],
 ['2025-05-28 17:29:10.807800', 37.775021763143954, -122.41925138066212],
 ['2025-05-28 17:29:11.807800', 37.77505142598227, -122.41922062383276],
 ['2025-05-28 17:29:12.807800', 37.77507703433093, -122.41919888159342],
 ['2025-05-28 17:29:13.807800', 37.77510516434602, -122.41917060363456],
 ['2025-05-28 17:29:14.807800', 37.775126872017125, -122.4191260656512],
 ['2025-05-28 17:29:15.807800', 37.7751538209434, -122.41907889008576],
 ['2025-05-28 17:29:16.807800', 37.77517027305343, -122.41905710918527],
 ['2025-05-28 17:29:17.807800', 37.77518176977941, -122.41901704365172],
 ['2025-05-28 17:29:18.807800', 37.77519087583205, -122.41899357935952],
 ['2025-05-28 17:29:19.807800', 37.77519728153923, 

In [7]:
from utils.location_functions import predict_future_dist_with_history
result = predict_future_dist_with_history(
    history_20, target_long, target_lat, 
    time_interval
)

future_lon, future_lat, dist_traveled, percent_done = result

print("Prediction Results:")
print(f"Future Position: ({future_lat:.6f}, {future_lon:.6f})")
print(f"Distance Traveled: {dist_traveled:.2f} meters")
print(f"Trip Completion: {percent_done:.2f}%")

Prediction Results:
Future Position: (37.776731, -122.417097)
Distance Traveled: 214.95 meters
Trip Completion: 23.82%


In [16]:
history_20[-1]

['2025-05-28 17:29:25.807800', 37.77535858852825, -122.4188216072092]

In [18]:
history_20[-1][1]

37.77535858852825

## vizulize on a map

In [ ]:
import folium

# Create coordinates from the dataframe for the original route
coordinates = [[row['latitude'], row['longitude']] for _, row in df.iterrows()]

# Calculate center point for the map
center_lat = (current_lat + target_lat) / 2
center_lon = (current_long + target_long) / 2

# Create a map for visualizing the prediction
m_prediction = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add the original route
folium.PolyLine(
    locations=coordinates,
    color='blue',
    weight=2,
    opacity=0.8,
    popup="Original Route"
).add_to(m_prediction)

# Add current position marker (start)
folium.Marker(
    location=[current_lat, current_long],
    icon=folium.Icon(color='green', icon='play')
).add_to(m_prediction)

# Add target position marker (end)
folium.Marker(
    location=[target_lat, target_long],
    popup=f"Target Position\nLat: {target_lat:.6f}\nLon: {target_long:.6f}",
    icon=folium.Icon(color='red', icon='stop')
).add_to(m_prediction)

# Add last known position marker
folium.Marker(
    location=[history_20[-1][1], history_20[-1][2]],
    icon=folium.Icon(color='purple', icon='circle')
).add_to(m_prediction)

# Add predicted future position
folium.Marker(
    location=[future_lat, future_lon],
    popup=f"Predicted Position after {time_interval}s\nLat: {future_lat:.6f}\nLon: {future_lon:.6f}\nDistance: {dist_traveled:.2f}m\nProgress: {percent_done:.1f}%",
    icon=folium.Icon(color='purple', icon='star')
).add_to(m_prediction)

# Add line from current to predicted position
folium.PolyLine(
    locations=[[current_lat, current_long], [future_lat, future_lon]],
    color='orange',
    weight=3,
    opacity=0.8,
    dash_array='10, 5',
).add_to(m_prediction)

# Add line from current to target (direct path)
folium.PolyLine(
    locations=[[current_lat, current_long], [target_lat, target_long]],
    color='gray',
    weight=1,
    opacity=0.5,
    dash_array='5, 5',
    popup="Direct path to target"
).add_to(m_prediction)

# Display the map
m_prediction